In [1]:
import pandas as pd
import numpy as np
from utils import *

In [2]:
data = pd.read_csv('num_data_del.csv')
data

,id,num_like_post,num_comment_post,num_share_post,label
0,1,19477,378,173,0
1,3,48,5,19,0
2,4,3,0,0,0
3,5,775,0,54,0
4,6,2,1,0,0
...,...,...,...,...,...
4219,4368,5800,1300,12000,0
4220,4369,21,1,0,0
4221,4370,3,1,0,1
4222,4371,144,38,87,1


In [3]:
cols = ['num_like_post', 'num_comment_post', 'num_share_post', 'label']
subnets = data[cols]
subnets.corr(method='pearson').style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)

,num_like_post,num_comment_post,num_share_post,label
num_like_post,1.00,0.01,-0.01,-0.01
num_comment_post,0.01,1.00,0.69,0.02
num_share_post,-0.01,0.69,1.00,0.07
label,-0.01,0.02,0.07,1.00


In [4]:
from preprocessing import *

In [5]:
test_string = "toản tỏan tỏa toả quả qủa quế qúê"
print(test_string)
standardize_vi_sentence_accents(test_string)

toản tỏan tỏa toả quả qủa quế qúê


'toản toản tỏa tỏa quả quả quế quế'

In [6]:
test_mail = "khai@gmail.com vcsdl@gd.sr.hk bdf@ydf.df.com"
test_link = "https:\\med.com.vn  www.hoag.com.vn uet.vnu.vn"
test_emoji_emotion = "😂😂 :v :) ^_^ ^-^ @_@"
print(replace_email(test_mail))
print(replace_link(test_link))
print(convert_emojis(test_emoji_emotion))
print(convert_emotions(test_emoji_emotion))

EMAIL EMAIL EMAIL
https:\URL
:face_with_tears_of_joy::face_with_tears_of_joy: :v :) ^_^ ^-^ @_@
😂😂 :v Happy_face_or_smiley Joyful ^-^ @_@


In [7]:
stopwords_file = "vietnamese-stopwords/vietnamese-stopwords.txt"
#teencode_file = "dict_topic_word/teencode.txt"

stopwords = get_stop_words(stopwords_file)
#teencode_dict = get_teencode_dict(teencode_file)

In [8]:
from vncorenlp import VnCoreNLP
annotator = VnCoreNLP("VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

In [9]:
cat_data = pd.read_csv('cat_data_del_down.csv')
cat_data

,id,post_message,label
0,439,Bộ Tài chính vừa có công văn đề nghị Bộ Tư phá...,0
1,3630,🏞 Dự án Sân golf Việt Yên:\n- Diện tích khu đấ...,0
2,4252,"😋🤤 VỚI NHÀ HÀNG, QUÁN ĂN THÌ CHỈ YÊU CẦU SÁT K...",0
3,673,Vingroup đầu tư 'siêu máy tính' AI đầu tiên tạ...,0
4,391,"Bắt đầu từ thứ Hai ngày 4/5 tới, xe buýt tại H...",0
...,...,...,...
1451,4336,"TQ 1980 có GDP 190 tỷ USD, thua VN bây giờ. Sa...",1
1452,4343,Virus Corona là vũ khí sinh học? Hiện nay có n...,1
1453,4364,Tin từ Phó Chủ Tịch UBND TPHCM Lê Thanh Liêm.\...,1
1454,4370,Lời cảnh tỉnh cho các thanh niên dân TỔ...tốc ...,1


In [10]:
def text_preprocessing(text, annotator=annotator, stopwords=[]):
    # kwargs: annotator, stopwords
  origin_text = copy.deepcopy(text)  
  try:
    text = unicodedata.normalize("NFC", text)
    text = text.lower()

    # Chuẩn hóa các từ có kí tự bị lặp lại
    text = remove_dup_char(text)

    text = replace_email(text)
    text = replace_emoji(text)

    # Thay thế tất cả các đường dẫn trong văn bản.
    text = replace_link(text)
    
    #remove punctuation
    table = str.maketrans('', '',string.punctuation)
    text = text.translate(table)

    # Tách mix từ và số
    text = split_alphanum(text)
    # Loại bỏ toàn bộ các ký tự đứng 1 mình
    text = strip_short(text, minsize=2)
    # Loại bỏ hết các số trong văn bản
    text = strip_numeric(text)

    # Loại bỏ các kí tự lạ như ≥
    text = strip_non_alphanum(text)
    # Chuẩn hóa cách đặt dấu cho các từ trong câu: ví dụ "nhân qủa" -> "nhân quả" 
    text = standardize_vi_sentence_accents(text)
    

    text = text.strip()
    if not text:
      return "EMPTY_STRING"
    #tokenize
    # luôn chỉ có 1 phần từ nên lấy [0]
    # lý do chỉ có 1 phần tử là vì ta đã xóa hết dấu câu cả văn bản coi là 1 câu
    # không còn dấu "." nên bộ tokenize xem như là 1 câu nên chỉ có 1 phần tử.
    list_token = annotator.tokenize(text)[0]

    # Chuẩn hóa teencode
    # list_token = [teencode_dict.get(token, token) for token in list_token]

    # remove stopword
    list_token = [token for token in list_token if token not in stopwords]

    norm_text = " ".join(list_token)

    return norm_text
  except:
    print("EXCEPTION with", origin_text, '\n')
    return "EXCEPTION_STRING"


In [11]:
my_text_preprocessing(cat_data.post_message[4], annotator=annotator, stopwords=stopwords)

'hai xe_buýt hà_nội hoạt_động lượt quy_định phòng_dịch'

In [12]:
text_preprocessing(cat_data.post_message[5], annotator=annotator, stopwords=stopwords)

'hơi muộn chúc anhchị nhà_báo báo_chí vui_vẻ sức_khoẻ hưởng_ứng kính mong mời anhchị nhà_báo chủ_đề cảnh khủng_hoảng hoang_dã hứng_thú tham_dự chi_tiết url'

In [13]:
cat_data['post_message_preproced'] = cat_data.post_message.apply(text_preprocessing, stopwords= stopwords, annotator= annotator)


In [14]:
cat_data

,id,post_message,label,post_message_preproced
0,439,Bộ Tài chính vừa có công văn đề nghị Bộ Tư phá...,0,tài_chính công_văn đề_nghị tư_pháp thẩm_định d...
1,3630,🏞 Dự án Sân golf Việt Yên:\n- Diện tích khu đấ...,0,emoji dự_án sân golf việt_yên diện_tích khu đấ...
2,4252,"😋🤤 VỚI NHÀ HÀNG, QUÁN ĂN THÌ CHỈ YÊU CẦU SÁT K...",0,emoji emoji nhà_hàng quán sát_khuẩn đồng quy_đ...
3,673,Vingroup đầu tư 'siêu máy tính' AI đầu tiên tạ...,0,vingroup đầu_tư siêu máy_tính việt_nam triệu t...
4,391,"Bắt đầu từ thứ Hai ngày 4/5 tới, xe buýt tại H...",0,hai xe_buýt hà_nội hoạt_động lượt quy_định phò...
...,...,...,...,...
1451,4336,"TQ 1980 có GDP 190 tỷ USD, thua VN bây giờ. Sa...",1,tq gdp tỷ usd thua vn gdp tỷ usd lịch_sử ca gi...
1452,4343,Virus Corona là vũ khí sinh học? Hiện nay có n...,1,virus corona vũ_khí_sinh_học bằng_chứng virus ...
1453,4364,Tin từ Phó Chủ Tịch UBND TPHCM Lê Thanh Liêm.\...,1,phó chủ_tịch ubnd tphcm lê_thanh_liêm tp hồ_ch...
1454,4370,Lời cảnh tỉnh cho các thanh niên dân TỔ...tốc ...,1,cảnh_tỉnh thanh_niên dân tổtốc độ bàn_thờ emoj...
